Installing required packages and dependencies

In [ ]:
%%capture
!pip install py-elvis
!pip install pyyaml==5.4
!pip install plotly==5.9.0
!pip install -U kaleido

!pip install stable-baselines3[extra]
!pip install stable-baselines
!pip install sb3-contrib
!pip install gym
!pip install -q wandb

In [ ]:
#Cloning repository and changing directory
!git clone https://github.com/francescomaldonato/RL_VPP_Thesis.git
%cd RL_VPP_Thesis/
%ls

In [ ]:
import yaml
import torch
from gym import Env
from VPP_environment import VPPEnv, VPP_Scenario_config
from elvis.config import ScenarioConfig
import os
import wandb
from wandb.integration.sb3 import WandbCallback
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from sb3_contrib import TRPO #The available algoritmhs in sb3-contrib for the custom environment with MultiInputPolicy
from sb3_contrib.common.maskable.utils import get_action_masks
import stable_baselines3 as sb3
from stable_baselines3.common.env_checker import check_env
import random

#Check if cuda device is available for training
print("Torch-Cuda available device:", torch.cuda.is_available())
print(sb3.get_system_info())
!wandb --version

Torch-Cuda available device: True
OS: Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic #1 SMP Fri Aug 26 08:44:51 UTC 2022
Python: 3.7.14
Stable-Baselines3: 1.6.1
PyTorch: 1.12.1+cu113
GPU Enabled: True
Numpy: 1.21.6
Gym: 0.21.0

({'OS': 'Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic #1 SMP Fri Aug 26 08:44:51 UTC 2022', 'Python': '3.7.14', 'Stable-Baselines3': '1.6.1', 'PyTorch': '1.12.1+cu113', 'GPU Enabled': 'True', 'Numpy': '1.21.6', 'Gym': '0.21.0'}, 'OS: Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic #1 SMP Fri Aug 26 08:44:51 UTC 2022\nPython: 3.7.14\nStable-Baselines3: 1.6.1\nPyTorch: 1.12.1+cu113\nGPU Enabled: True\nNumpy: 1.21.6\nGym: 0.21.0\n')
wandb, version 0.13.3


In [ ]:
# Ensure deterministic behavior
torch.backends.cudnn.deterministic = True
random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)

In [ ]:
#Loading paths for input data
current_folder = ''
VPP_data_input_path = current_folder + 'data/data_testing/environment_table/' + 'Environment_data_2020.csv'
elvis_input_folder = current_folder + 'data/config_builder/'

case = 'wohnblock_household_simulation_adaptive.yaml'
with open(elvis_input_folder + case, 'r') as file:
    yaml_str = yaml.full_load(file)

elvis_config_file = ScenarioConfig.from_yaml(yaml_str)
VPP_config_file = VPP_Scenario_config(yaml_str)

print(elvis_config_file)
print(VPP_config_file)

Vehicle types: <generator object ScenarioConfig.__str__.<locals>.<genexpr> at 0x7f9456e8c5d0>Mean parking time: 23.9
Std deviation of parking time: 1
Mean value of the SOC distribution: 0.5
Std deviation of the SOC distribution: 0.1
Max parking time: 24
Number of charging events per week: 20
Vehicles are disconnected only depending on their parking time
Queue length: 0
Opening hours: None
Scheduling policy: Uncontrolled

{'start_date': '2022-01-01T00:00:00', 'end_date': '2023-01-01T00:00:00', 'resolution': '0:15:00', 'num_households': 4, 'solar_power': 16, 'wind_power': 12, 'EV_types': [{'battery': {'capacity': 100, 'efficiency': 1, 'max_charge_power': 150, 'min_charge_power': 0}, 'brand': 'Tesla', 'model': 'Model S', 'probability': 1}], 'charging_stations_n': 4, 'EVs_n': 20, 'EVs_n_max': 1044, 'mean_park': 23.9, 'std_deviation_park': 1, 'EVs_mean_soc': 50.0, 'EVs_std_deviation_soc': 10.0, 'EV_load_max': 44, 'EV_load_rated': 14.8, 'EV_load_min': 1, 'houseRWload_max': 10, 'av_max_energy

In [ ]:
#Environment initialization
env = VPPEnv(VPP_data_input_path, elvis_config_file, VPP_config_file)
env.plot_ELVIS_data()

Charging event: 1, Arrival time: 2022-01-01 11:15:00, Parking_time: 21.919598287704858, Leaving_time: 2022-01-02 09:10:10.553836, SOC: 0.5184766221383362, SOC target: 1.0, Connected car: Tesla, Model S 
 ... 
 Charging event: 1043, Arrival time: 2022-12-31 21:30:00, Parking_time: 24, Leaving_time: 2023-01-01 21:30:00, SOC: 0.4123731400954075, SOC target: 1.0, Connected car: Tesla, Model S 

-DATASET: House&RW_energy_sum=kWh  -21214.64 , over-consume=kWh  4947.18 , under-consume=kWh  -26161.81 , Total_cost=€  -489.75 , overcost=€  233.11
- ELVIS.Simulation (Av.EV_SOC=  50.0 %):
 Sum_Energy=kWh  21993.56 , over-consume=kWh  38771.58 , under-consume=kWh  16778.02 , Total_cost=€  898.59 , overcost=€  1376.31 , Charging_events=  1043 
- Exp.VPP_goals: Energy_consumed=kWh 0, Av.load=kW 0, Std.load=kW 0, Total_cost=€ 0 , Av.EV_en_left=kWh  75.08


In [ ]:
#Function to check custom environment and output additional warnings if needed
check_env(env)
env.plot_reward_functions()

- ELVIS.Simulation (Av.EV_SOC=  50.0 %):
 Sum_Energy=kWh  21170.11 , over-consume=kWh  38536.58 , under-consume=kWh  17366.48 , Total_cost=€  878.48 , overcost=€  1381.24 , Av.EV_en_left=kWh  100.0 , Charging_events=  1043 
- Exp.VPP_goals: Energy_consumed=kWh 0, Av.load=kW 0, Std.load=kW 0, Total_cost=€ 0 , Av.EV_en_left=kWh  75.08
Simulating VPP....


In [ ]:
TRPO_path = "trained_models/TRPO_models/TRPO"

#In Colab, uncomment below:
%env "WANDB_DISABLE_CODE" True
%env "WANDB_NOTEBOOK_NAME" "Hyperparameters_sweep_notebooks/TRPO_VPP_Hyperp_Sweep.ipynb"
os.environ['WANDB_NOTEBOOK_NAME'] = 'Hyperparameters_sweep_notebooks/TRPO_VPP_Hyperp_Sweep.ipynb'
wandb.login(relogin=True)

#In local notebook, uncomment below:
#your_wandb_login_code = 0123456789abcdefghijklmnopqrstwxyzàèìòù0 #example length
#!wandb login {your_wandb_login_code}

In [ ]:
sweep_config = {
    "name": "TRPO-sweep",
    "project": "RL_VPP_Thesis",
    #"entity" : "user_avocado",

    "method" : "random",
    "metric": {"name": "cumulative_reward",
                #"target": 10000000000,
                "goal": "maximize"},
    "early_terminate":{    
        "type": "hyperband",
        "min_iter": 10,
        "eta": 3
        },

    "parameters": {
        "policy_type": {
            "value": "MultiInputPolicy"
            },
        "epochs": {
            "value": 1,
            },
        
        "batch_size": {
            #"value" : 2920
            "values" : [2920, 8760, 11680, 17520]
            },

        "total_timesteps": {
            "value": 500000,
            },
        "learning_rate": {
            "min": 0.00001,
            "max": 0.1,
            "distribution": "log_uniform_values"
            },
        # "lr_schedule": {
        #     "value": "constant",
        #     #"values": ["linear", "constant"]
        #     },
        "gamma": { #discount factor
            "max": 0.9999,
            "min": 0.89,
            "distribution": "log_uniform_values"
            },
        "gae_lambda": {
            "values": [0.8, 0.9, 0.92, 0.95, 0.98, 0.99, 1.0]
            },

        "line_search_shrinking_factor": {
            "values": [0.6, 0.7, 0.8, 0.9]
          },
        "line_search_max_iter": {
          "value" : 10
          },
        "n_critic_updates": {
            "values":  [5, 10, 20, 25, 30],
          },
        "cg_max_steps": {
            "values":  [5, 10, 20, 25, 30],
          },
        "target_kl": {
            "values": [0.1, 0.05, 0.03, 0.02, 0.01, 0.005, 0.001],
          },

        "ortho_init": {
            #"values": [True, False]
            "value": True
            },
        # "use_sde":{
        #     "values": [True, False]
        #   },
        "net_arch": {
            #"values": ["small-short", "small-long", "medium-short", "medium", "big-short", "big", "big-large"],
            "values": ["small-separate", "small-shared", "medium-short", "medium", "medium-separate", "big-shared", "big", "big-separate"]
            },
        "activation_fn":{
            "value": "Tanh",
            #"values": ["Tanh", "ReLU"],
            #"values": ["Tanh", "ReLU", "ELU"],
            #"values": ["Tanh", "ReLU", "ELU", "LeakyReLU"]
            },
        # "optimizer_class": {
        #     "values": ["RMSprop", "Adam", "SGD"]
        #     }
    }
}

policy_dict = {
    "net_arch": {
        "small-separate": [dict(pi=[64, 64, 64], vf=[64, 64, 64])],
        "small-shared": [128, dict(pi=[64, 64], vf=[64, 64])],
        
        "medium-short": [dict(pi=[128, 128], vf=[128, 128])],
        "medium": [64, dict(pi=[256, 256], vf=[256, 256])],
        "medium-separate": [dict(pi=[256, 256], vf=[256, 256])],

        "big-shared": [256, dict(pi=[64, 64], vf=[64, 64])],
        "big": [128, dict(pi=[512, 512], vf=[512, 512])],
        "big-separate": [dict(pi=[512, 512], vf=[512, 512])],
        },
        
    "activation_fn":{
        "Tanh": torch.nn.modules.activation.Tanh,
        "ReLU": torch.nn.modules.activation.ReLU,
        "ELU": torch.nn.modules.activation.ELU,
        "LeakyReLU": torch.nn.modules.activation.LeakyReLU
        },

    "optimizer_class": {
        "RMSprop": torch.optim.RMSprop,
        "Adam": torch.optim.Adam,
        "SGD": torch.optim.SGD
        }
}

#to enable adjustable learning rate
from typing import Callable, Union
def linear_schedule(initial_value: Union[float, str]) -> Callable[[float], float]:
    if isinstance(initial_value, str):
        initial_value = float(initial_value)
    def func(progress_remaining: float) -> float:
        return progress_remaining * initial_value
    return func

#Trained model testing function
def evaluate_model(env, model, wandb_run_dir, deterministic=True):
    obs = env.reset()
    done = False
    cumulative_reward = 0
    # cell and hidden state of the LSTM
    lstm_states = None
    num_envs = 1
    # Episode start signals are used to reset the lstm states
    episode_starts = [True]
    while not done:
        # Retrieve current action mask
        action_masks = get_action_masks(env)
        action, lstm_states = model.predict(obs, state=lstm_states, episode_start=episode_starts, deterministic = deterministic) #Now using our trained model with deterministic prediction [should improve performances]
        #env.lstm_state = lstm_states
        obs, reward, done, info = env.step(action)
        episode_starts = done
        cumulative_reward+=reward
    VPP_results_plot = env.plot_VPP_results()
    VPP_results_plot.show()
    VPP_results_plot.write_image(os.path.join(wandb_run_dir, "VPP_results_plot.png"))
    plot_VPP_energies = env.plot_VPP_energies()
    plot_VPP_energies.show()
    plot_VPP_energies.write_image(os.path.join(wandb_run_dir, "VPP_energies_plot.png"))
    plot_rewards_results = env.plot_rewards_results()
    plot_rewards_results.show()
    plot_rewards_results.write_image(os.path.join(wandb_run_dir, "VPP_rewards_plot.png"))
    plot_rewards_stats = env.plot_rewards_stats()
    plot_rewards_stats.show()
    plot_rewards_stats.write_image(os.path.join(wandb_run_dir, "VPP_rewards_stats_plot.png"))
    plot_EVs_kpi = env.plot_EVs_kpi()
    plot_EVs_kpi.show()
    plot_EVs_kpi.write_image(os.path.join(wandb_run_dir, "EVs_kpi_plot.png"))
    plot_actions_kpi = env.plot_actions_kpi()
    plot_actions_kpi.show()
    plot_actions_kpi.write_image(os.path.join(wandb_run_dir, "actions_kpi_plot.png"))
    plot_load_kpi = env.plot_load_kpi()
    plot_load_kpi.show()
    plot_load_kpi.write_image(os.path.join(wandb_run_dir, "load_kpi_plot.png"))
    plot_yearly_log_kpi = env.plot_yearly_load_log()
    plot_yearly_log_kpi.show()
    plot_yearly_log_kpi.write_image(os.path.join(wandb_run_dir, "yearly_log_load_plot.png"))
    plot_comparison = env.plot_VPP_Elvis_comparison()
    plot_comparison.show()
    plot_comparison.write_image(os.path.join(wandb_run_dir, "Elvis_VPP_comparison_plot.png"))
    VPP_table = env.save_VPP_table(save_path=os.path.join(wandb_run_dir,'VPP_table.csv'))
    #wandb.log({"VPP_table": VPP_table})

    wandb.save(f"VPP_results_plot.png")
    wandb.save(f"VPP_energies_plot.png")
    wandb.save(f"VPP_rewards_plot.png")
    wandb.save(f"VPP_rewards_stats_plot.png")
    wandb.save(f"EVs_kpi_plot.png")
    wandb.save(f"actions_kpi_plot.png")
    wandb.save(f"load_kpi_plot.png")
    wandb.save(f"yearly_log_load_plot.png")
    wandb.save(f"Elvis_VPP_comparison_plot.png")
    wandb.save(f"VPP_table.csv")
    # wandb.log({"VPP_results_plot": wandb.Image("VPP_results_plot.png")})
    return cumulative_reward, env.av_EV_energy_left, env.overconsumed_en, env.underconsumed_en, env.sim_overcost

In [ ]:
#ENVIRONMENT MASKING (TO_DO?)
#InvalidActionEnvMultiDiscrete
#ENVIRONMENT WRAPPING
X_env = Monitor(env)
#Vectorized environment wrapper
X_env = DummyVecEnv([lambda: X_env])
#Sync custom tensorboard patch
#wandb.tensorboard.patch(root_logdir='wandb/tensorboard_log/', pytorch=True)
tensorboard_log_path = "wandb/tensorboard_log/"

#Model training function for Hyperparameters Sweep
def train_func():
    with wandb.init(#job_type='TRPO_trained_model',
                    reinit=True,
                    #settings=wandb.Settings(symlink=False), reinit=False
                    sync_tensorboard=True,
                    monitor_gym=False, save_code=False) as run:
        config = wandb.config
        policy_kwargs =  dict(
            ortho_init = config["ortho_init"],
            net_arch = policy_dict["net_arch"][config["net_arch"]],
            activation_fn = policy_dict["activation_fn"][config["activation_fn"]],
            #optimizer_class = policy_dict["optimizer_class"][config["optimizer_class"]]
        )
        # if config["lr_schedule"] == "linear":
        #     learning_rate = linear_schedule(config["learning_rate"])
        # elif config["lr_schedule"] == "costant": learning_rate = config["learning_rate"]

        #model definition
        model = TRPO(config["policy_type"], X_env,
                    learning_rate = config["learning_rate"],
                    n_steps = config["batch_size"],
                    batch_size = config["batch_size"],
                    #n_epochs = config["n_epochs"],
                    gamma = config["gamma"],
                    gae_lambda = config["gae_lambda"],
                    #clip_range = config["clip_range"],
                    cg_max_steps = config["cg_max_steps"],
                    n_critic_updates = config["n_critic_updates"],
                    line_search_shrinking_factor = config["line_search_shrinking_factor"],
                    line_search_max_iter = config["line_search_max_iter"],
                    target_kl = config["target_kl"],
                    #use_sde = config["use_sde"],
                    #create_eval_env = False,
                    policy_kwargs = policy_kwargs,
                    verbose=0,
                    #tensorboard_log= os.path.join(tensorboard_log_path,f'TRPO_{run.id}_1')
                    tensorboard_log = tensorboard_log_path
                    )
        
        #Training of each hyperprameter set    
        for epoch in range(config["epochs"]):
            model.learn(total_timesteps=config["total_timesteps"],
                tb_log_name=f'TRPO_{run.id}',
                callback=WandbCallback(
                    gradient_save_freq=10000,
                    #model_save_path=f"trained_models/PPO_sweep_{run.id}",
                    verbose=1)
                )
            ##TESTING trained model
            #cumulative_reward, std_reward = evaluate_policy(model, X_env, n_eval_episodes=1, render=False)
            cumulative_reward, av_EV_energy_left, overconsume, underconsume, overcost = evaluate_model(env, model, wandb.run.dir, deterministic=True)
            ##Logging results
            wandb.log({"cumulative_reward":cumulative_reward, "av_EV_energy_left":av_EV_energy_left, "overconsume":overconsume, "underconsume":underconsume, "overcost":overcost})
            #model.save(current_folder + PPO_path + f"_{run.id}")
            model.save(os.path.join(wandb.run.dir, f"model_TRPO_{run.id}"))
            wandb.save(f"model_TRPO_{run.id}")       
        # Sync wandb
        #wandb.save(f'wandb/tensorboard_log/TRPO_{run.id}_1')
        run.finish()
            
        #print(f"END OF TRAINING #### Model reward: {cumulative_reward}; Training complete.")

In [ ]:
count = 50 # number of Training runs to execute

sweep_id = wandb.sweep(sweep_config)
wandb.agent(sweep_id, train_func, count=count)

#!wandb sync log_dir
!wandb sync wandb/tensorboard_log/

In [ ]:
env.close()
#run.finish()
wandb.finish()